# 使用 LangChain、GPT 和 Activeloop 的 Deep Lake 处理代码库
在本教程中，我们将使用 Langchain + Activeloop 的 Deep Lake 和 GPT 来分析 LangChain 自身的代码库。

## 设计

1. 准备数据：
   1. 使用 `langchain_community.document_loaders.TextLoader` 上传所有 Python 项目文件。我们将这些文件称为**文档**。
   2. 使用 `langchain_text_splitters.CharacterTextSplitter` 将所有文档分割成块。
   3. 使用 `langchain.embeddings.openai.OpenAIEmbeddings` 和 `langchain_community.vectorstores.DeepLake` 对块进行嵌入并将其上传到 DeepLake
2. 问答：
   1. 从 `langchain.chat_models.ChatOpenAI` 和 `langchain.chains.ConversationalRetrievalChain` 构建链
   2. 准备问题。
   3. 运行链获取答案。


## 实现

### 集成准备

我们需要设置外部服务的密钥并安装必要的 Python 库。

In [ ]:
#!python3 -m pip install --upgrade langchain langchain-deeplake openai

设置 OpenAI 嵌入、Deep Lake 多模态向量存储 API 并进行身份认证。

完整的 Deep Lake 文档请访问 https://docs.activeloop.ai/ 和 API 参考 https://docs.deeplake.ai/en/latest/

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()
# 请手动输入 OpenAI 密钥

如果你想创建并发布自己的数据集，请登录 Deep Lake。你可以从平台 [app.activeloop.ai](https://app.activeloop.ai) 获取 API 密钥

In [2]:
activeloop_token = getpass("Activeloop Token:")
os.environ["ACTIVELOOP_TOKEN"] = activeloop_token

### 准备数据

加载所有仓库文件。这里我们假设这个笔记本是作为 langchain fork 的一部分下载的，我们处理 `langchain` 仓库的 Python 文件。

如果你想使用来自不同仓库的文件，请将 `root_dir` 更改为你的仓库的根目录。

In [10]:
!ls "../../../../../../libs"

CITATION.cff  MIGRATE.md  README.md  libs	  poetry.toml
LICENSE       Makefile	  docs	     poetry.lock  pyproject.toml


In [11]:
from langchain_community.document_loaders import TextLoader

root_dir = "../../../../../../libs"

docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith(".py") and "*venv/" not in dirpath:
            try:
                loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                docs.extend(loader.load_and_split())
            except Exception:
                pass
print(f"{len(docs)}")

2554


然后，对文件进行分块

In [12]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
print(f"{len(texts)}")

Created a chunk of size 1010, which is longer than the specified 1000
Created a chunk of size 3466, which is longer than the specified 1000
Created a chunk of size 1375, which is longer than the specified 1000
Created a chunk of size 1928, which is longer than the specified 1000
Created a chunk of size 1075, which is longer than the specified 1000
Created a chunk of size 1063, which is longer than the specified 1000
Created a chunk of size 1083, which is longer than the specified 1000
Created a chunk of size 1074, which is longer than the specified 1000
Created a chunk of size 1591, which is longer than the specified 1000
Created a chunk of size 2300, which is longer than the specified 1000
Created a chunk of size 1040, which is longer than the specified 1000
Created a chunk of size 1018, which is longer than the specified 1000
Created a chunk of size 2787, which is longer than the specified 1000
Created a chunk of size 1018, which is longer than the specified 1000
Created a chunk of s

8244


然后对块进行嵌入并将它们上传到 DeepLake。

这可能需要几分钟时间。

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base='', openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='', openai_organization='', allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6, request_timeout=None, headers=None, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={})

In [ ]:
from langchain_deeplake.vectorstores import DeeplakeVectorStore

username = "<USERNAME_OR_ORG>"


db = DeeplakeVectorStore.from_documents(
    documents=texts,
    embedding=embeddings,
    dataset_path=f"hub://{username}/langchain-code",
    overwrite=True,
)
db

### 问答
首先加载数据集，构建检索器，然后构建对话链

In [ ]:
db = DeeplakeVectorStore(
    dataset_path=f"hub://{username}/langchain-code",
    read_only=True,
    embedding_function=embeddings,
)

In [18]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 20
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 20

In [20]:
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0613")  # 'ada' 'gpt-3.5-turbo-0613' 'gpt-4',
qa = RetrievalQA.from_llm(model, retriever=retriever)

In [32]:
questions = [
    "What is the class hierarchy?",
    "What classes are derived from the Chain class?",
    "What kind of retrievers does LangChain have?",
]
chat_history = []
qa_dict = {}

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    qa_dict[question] = result["answer"]
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What is the class hierarchy? 

**Answer**: The class hierarchy for Memory is as follows:

    BaseMemory --> BaseChatMemory --> <name>Memory  # Examples: ZepMemory, MotorheadMemory

The class hierarchy for ChatMessageHistory is as follows:

    BaseChatMessageHistory --> <name>ChatMessageHistory  # Example: ZepChatMessageHistory

The class hierarchy for Prompt is as follows:

    BasePromptTemplate --> PipelinePromptTemplate
                           StringPromptTemplate --> PromptTemplate
                                                    FewShotPromptTemplate
                                                    FewShotPromptWithTemplates
                           BaseChatPromptTemplate --> AutoGPTPrompt
                                                      ChatPromptTemplate --> AgentScratchPadChatPromptTemplate
 

-> **Question**: What classes are derived from the Chain class? 

**Answer**: The classes derived from the Chain class are:

- APIChain
- OpenAPIEndpoin

In [31]:
qa_dict

{'question': 'LangChain possesses a variety of retrievers including:\n\n1. ArxivRetriever\n2. AzureAISearchRetriever\n3. BM25Retriever\n4. ChaindeskRetriever\n5. ChatGPTPluginRetriever\n6. ContextualCompressionRetriever\n7. DocArrayRetriever\n8. ElasticSearchBM25Retriever\n9. EnsembleRetriever\n10. GoogleVertexAISearchRetriever\n11. AmazonKendraRetriever\n12. KNNRetriever\n13. LlamaIndexGraphRetriever\n14. LlamaIndexRetriever\n15. MergerRetriever\n16. MetalRetriever\n17. MilvusRetriever\n18. MultiQueryRetriever\n19. ParentDocumentRetriever\n20. PineconeHybridSearchRetriever\n21. PubMedRetriever\n22. RePhraseQueryRetriever\n23. RemoteLangChainRetriever\n24. SelfQueryRetriever\n25. SVMRetriever\n26. TFIDFRetriever\n27. TimeWeightedVectorStoreRetriever\n28. VespaRetriever\n29. WeaviateHybridSearchRetriever\n30. WebResearchRetriever\n31. WikipediaRetriever\n32. ZepRetriever\n33. ZillizRetriever\n\nIt also includes self query translators like:\n\n1. ChromaTranslator\n2. DeepLakeTranslator\n

In [33]:
print(qa_dict["What is the class hierarchy?"])

The class hierarchy for Memory is as follows:

    BaseMemory --> BaseChatMemory --> <name>Memory  # Examples: ZepMemory, MotorheadMemory

The class hierarchy for ChatMessageHistory is as follows:

    BaseChatMessageHistory --> <name>ChatMessageHistory  # Example: ZepChatMessageHistory

The class hierarchy for Prompt is as follows:

    BasePromptTemplate --> PipelinePromptTemplate
                           StringPromptTemplate --> PromptTemplate
                                                    FewShotPromptTemplate
                                                    FewShotPromptWithTemplates
                           BaseChatPromptTemplate --> AutoGPTPrompt
                                                      ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



In [34]:
print(qa_dict["What classes are derived from the Chain class?"])

The classes derived from the Chain class are:

- APIChain
- OpenAPIEndpointChain
- AnalyzeDocumentChain
- MapReduceDocumentsChain
- MapRerankDocumentsChain
- ReduceDocumentsChain
- RefineDocumentsChain
- StuffDocumentsChain
- ConstitutionalChain
- ConversationChain
- ChatVectorDBChain
- ConversationalRetrievalChain
- FlareChain
- ArangoGraphQAChain
- GraphQAChain
- GraphCypherQAChain
- HugeGraphQAChain
- KuzuQAChain
- NebulaGraphQAChain
- NeptuneOpenCypherQAChain
- GraphSparqlQAChain
- HypotheticalDocumentEmbedder
- LLMChain
- LLMBashChain
- LLMCheckerChain
- LLMMathChain
- LLMRequestsChain
- LLMSummarizationCheckerChain
- MapReduceChain
- OpenAIModerationChain
- NatBotChain
- QAGenerationChain
- QAWithSourcesChain
- RetrievalQAWithSourcesChain
- VectorDBQAWithSourcesChain
- RetrievalQA
- VectorDBQA
- LLMRouterChain
- MultiPromptChain
- MultiRetrievalQAChain
- MultiRouteChain
- RouterChain
- SequentialChain
- SimpleSequentialChain
- TransformChain
- TaskPlaningChain
- QueryChain
- CPAL

In [35]:
print(qa_dict["What kind of retrievers does LangChain have?"])

The LangChain class includes various types of retrievers such as:

- ArxivRetriever
- AzureAISearchRetriever
- BM25Retriever
- ChaindeskRetriever
- ChatGPTPluginRetriever
- ContextualCompressionRetriever
- DocArrayRetriever
- ElasticSearchBM25Retriever
- EnsembleRetriever
- GoogleVertexAISearchRetriever
- AmazonKendraRetriever
- KNNRetriever
- LlamaIndexGraphRetriever and LlamaIndexRetriever
- MergerRetriever
- MetalRetriever
- MilvusRetriever
- MultiQueryRetriever
- ParentDocumentRetriever
- PineconeHybridSearchRetriever
- PubMedRetriever
- RePhraseQueryRetriever
- RemoteLangChainRetriever
- SelfQueryRetriever
- SVMRetriever
- TFIDFRetriever
- TimeWeightedVectorStoreRetriever
- VespaRetriever
- WeaviateHybridSearchRetriever
- WebResearchRetriever
- WikipediaRetriever
- ZepRetriever
- ZillizRetriever
